# get pageviews with AWS

#### how to do jupyter with aws

- https://dataschool.com/data-modeling-101/running-jupyter-notebook-on-an-ec2-server/
    - except jupyter_notebook_config.py_ should be ...config.py instead
- https://gist.github.com/J535D165/0e840291e7b2598ec157e13e9b9ca569
    - trying this for how to use nohup
- some medium [article](https://medium.com/@christinakouride/a-beginners-guide-to-running-jupyter-notebook-on-amazon-ec2-69e1b74e73cc#:~:text=Your%20Jupyter%20Notebook%20server%20will,for%20time%20not%20using%20it.)
    - they pointed out that notebook will keep running, but didn't mention nohup

In [70]:
import os, requests, gzip, pickle, io
import pandas as pd
import datetime as dt
import mysql.connector as mysql, sqlalchemy

In [2]:
from platform import python_version
print(python_version())

3.9.7


In [4]:
mysql_user = 'ubuntu'
# mysql_pass = input(f'Enter the MySQL password for user {mysql_user}: ')

In [5]:
def connect_mariadb():
    host='localhost'; user=mysql_user; passwd=mysql_pass; dbname='jawiki';
    cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
    cur = cxn.cursor()

    connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
    try:
        engine = sqlalchemy.create_engine(connection_str)
        conn = engine.connect()
    except Exception as e:
        print('Database connection error - check creds')
        print(e)
    return cxn, cur, engine, conn
        
cxn, cur, engine, conn = connect_mariadb()

## get page_titles and urls

###### unpickle page list

In [6]:
with open('../data/processed/jawiki/' + 'disaster_descendants_raw.pickle', 'rb') as f:
    disaster_descendants_raw = pickle.load(f)

In [7]:
disaster_cat_page_ids = {'火山災害':2390743, '熱帯低気圧':626482, '雪害':2390774, '地震':135264, '津波':765772}  # '自然災害':137069, 
disasters_english = {'火山災害':'VolcanicDisaster', '熱帯低気圧':'TropicalCyclones', '雪害':'SnowDamage', '地震':'Earthquake', '津波':'Tsunami'}
disasters = list(disaster_cat_page_ids.keys())

In [8]:
def failed_decode(a):
    try:
        a.decode('utf-8')
    except UnicodeDecodeError:
        return True
    return False

def bytearray_to_str(a:bytearray, encoding='utf-8') -> str:
    if type(a) != bytearray:
        return a        
    while failed_decode(a):
        a = a[:-1]
    return str(a.decode(encoding))

In [9]:
d = {}
for i in disaster_descendants_raw:
    d[i] = (disaster_descendants_raw[i]
            .drop_duplicates(subset='id')
            .applymap(bytearray_to_str)
           )
    d[i] = d[i][d[i].namespace == 0]
    d[i]['page_title'] = d[i].name.map(lambda x: str(x).split(sep='\n')[-1])
disaster_descendants = d
del d

In [10]:
all_titles = [j for i in disaster_descendants for j in disaster_descendants[i].page_title]

## functions

##### function download_file

In [11]:
def download_file(url, dirpath='./'):
    local_filepath = dirpath + url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=131072):   #8192
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filepath
# https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests

###### function ```open_hour_file``` to provide the unzipped file to python

In [20]:
def open_hour_file(path:str):
    if path[-3:] == ".gz":
        return gzip.open(path, mode="rt", encoding="utf-8", errors="replace")
    else:
        return open(path, mode="rt", encoding="utf-8", errors="replace")
# https://github.com/mediawiki-utilities/python-mwviews/blob/main/src/mwviews/utilities/aggregate.py
# https://stackoverflow.com/questions/30582162/creating-a-missing-directory-file-structure-python

###### function subset_pageviews_by_project to get all jawiki pageviews

In [84]:
def parse_time_from_filename(filename:str):
    [d, t] = filename.split('-')[1:]
    return dt.datetime(int(d[:4]), int(d[4:6]), int(d[6:]), int(t[:2]), tzinfo=dt.timezone.utc)

In [95]:
def get_pageviews_subset_by_project(fpath_in, proj:str='ja', ) -> pd.DataFrame:
    lst = []
    colnames=['domain_code','page_title','count_views','total_response_size']
    # subset pageview records by project as list of strings
    with open_hour_file(fpath_in) as f_in:
        while (line := f_in.readline()):
            if line[:3] != proj + ' ':
                continue
            lst.append(line)
    # turn list of strings into dataframe
    df = pd.read_csv(
        io.StringIO('\n'.join(lst))
        ,delim_whitespace=True
        ,names=colnames)
    # add datetime as column based on filename
    parsed_datetime = parse_time_from_filename(fpath_in.split(sep='/')[-1])
    df = df.assign(datetime_viewed_UTC=parsed_datetime)
    now_datetime = dt.datetime.now(tz=dt.timezone.utc)
    df = df.assign(datetime_added_UTC=now_datetime)
    return df
    

In [96]:
df = get_pageviews_subset_by_project(out_dirpath + 'pageviews-20210101-000000.gz')

In [98]:
df.to_sql(name='pageviews',con=conn, if_exists='replace', flavor='mysql')

In [105]:
pd.read_sql('desc pageviews;', con=conn)

,Field,Type,Null,Key,Default,Extra
0,index,bigint(20),YES,MUL,None,
1,domain_code,text,YES,,None,
2,page_title,text,YES,,None,
3,count_views,bigint(20),YES,,None,
4,total_response_size,bigint(20),YES,,None,
5,datetime_viewed_UTC,timestamp,YES,,None,
6,datetime_added_UTC,timestamp,YES,,None,


In [103]:
pd.read_sql('select * from pageviews limit 10;', con=conn, index_col='index')

,domain_code,page_title,count_views,total_response_size,datetime_viewed_UTC,datetime_added_UTC
index,,,,,,
0,ja,"Hello,_world.",1,0,2021-01-01,2022-03-20 07:04:24
1,ja,$百萬BABY,1,0,2021-01-01,2022-03-20 07:04:24
2,ja,'81秋・全番組総出演!激唱!!オールスター,1,0,2021-01-01,2022-03-20 07:04:24
3,ja,(ry,1,0,2021-01-01,2022-03-20 07:04:24
4,ja,+Lhaca,1,0,2021-01-01,2022-03-20 07:04:24
5,ja,-,138,0,2021-01-01,2022-03-20 07:04:24
6,ja,.260レミントン,1,0,2021-01-01,2022-03-20 07:04:24
7,ja,.300ウィンチェスター・ショート・マグナム,2,0,2021-01-01,2022-03-20 07:04:24
8,ja,.308ウィンチェスター,1,0,2021-01-01,2022-03-20 07:04:24


###### function pageviews_to_sql

In [ ]:
def pageviews_to_sql(df:pd.DataFrame, conn):
    

## get the files

###### get many urls

In [ ]:
'https://dumps.wikimedia.org/other/pageviews/2021/2021-01/pageviews-20210101-000000.gz'

In [74]:
ys = list(range(2021,2022))
hour_strings = list(map(lambda x: str(x).zfill(2), range(0,24)))
dates = pd.date_range(dt.datetime(ys[0],1,1), 
                      end=dt.datetime(ys[-1]+1,1,1), 
                      freq='D')  # , tz='Japan'
hs = map(lambda x: str(x).zfill(2), range(0,24))
base_url = 'https://dumps.wikimedia.org/other/pageviews/'
out_dirpath = '../data/processed/pageviews/'

urls = [f'{base_url}{d.year}/{d.year}-{str(d.month).zfill(2)}'
        f'/pageviews-{d.year}{str(d.month).zfill(2)}{str(d.day).zfill(2)}-{h}0000.gz'
        for d in dates for h in hour_strings]
outpaths = [f'{out_dirpath}{d.year}/{d.year}-{str(d.month).zfill(2)}'
        f'/pageviews-{d.year}{str(d.month).zfill(2)}{str(d.day).zfill(2)}-{h}0000.txt'
        for d in dates for h in hour_strings]

##### populate the database

###### test to_sql

In [18]:
temp_fpath = download_file(url=urls[0], dirpath=out_dirpath)

###### get a page_title-filtered hourfile from txt as a dataframe

###### get a project-filtered  hourfile as txt, then df, then mariadb table

In [33]:
subset_pageviews_by_project(temp_fpath,out_dirpath+'test2.txt')

###### pageviews as df

In [56]:
df = pd.read_csv(out_dirpath+'test2.txt', sep=' ', 
                 names=['domain_code','page_title','count_views','total_response_size'])

In [53]:
some_time = dt.datetime(2021,1,1,0,0, tzinfo=dt.timezone.utc)

In [98]:
os.listdir('../data/processed/pageviews/2021/2021-01/')

[]

##### download many files (OBSOLETE, SWITCHING TO MARIADB ON EC2)

###### peek

In [ ]:
# !jupyter nbconvert --to script '1.22-sfb-get-pageviews-with-AWS-lambda.ipynb'

##### -----

###### ------

##### test running cell while laptop sleeping

###### idea

- consider the case where:
    - i start the following cell's script as follows:
        - on AWS from a ssh terminal 
        - via nohup jupyter on computer A
    - then i disconnect the ssh from computer A
        - so that the cell is still running in the instance
- then i find the result that:
    - i can reopen jupyter from another ssh terminal
    - i can interrupt the script
    - but i can't see its in-process results
- strategy for now:
    - develop the script on whatever computers
    - then run it from a browser tab in my android phone
        - that way i can keep "tabs" on it (😉)

###### test code

In [ ]:
from IPython.display import clear_output
import time

for i in range(600):
    print(i)
    time.sleep(1)
    clear_output(wait=True)

##### download many files (OBSOLETE, SWITCHING TO MARIADB ON EC2)

###### function ```subset_pageviews_by_page_titles``` to clean and filter pageview records

###### function ```subset_pageviews_by_project``` to clean and filter pageview records